In [546]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

#Import yfinance
import tensorflow as tf


In [553]:
#Downloads info from Yahoo Finance
df = yf.download(tickers="BTC-USD",
                            period="60d",
                            interval="15m",
                            auto_adjust=True)


#Sort values from past to present
df = df.reset_index(drop=True)

df = df.drop(columns=['Volume'])


#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


df = df.drop(['Low','Close','High'], axis=1)


df

[*********************100%***********************]  1 of 1 completed


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
0,21046.812500,21046.812500,21046.812500,21046.812500,0.000000,0.000000,0.000000,21069.736328,0.000000,100.000000,0.000000
1,21021.128906,21033.970703,21033.970703,21033.970703,0.000000,0.000000,0.000000,21069.632812,-0.008258,0.000000,0.000000
2,21073.480469,21047.140625,21047.140625,21047.140625,0.000000,0.000000,0.000000,21069.736328,-0.958187,0.000000,0.000000
3,21075.144531,21054.141602,21054.141602,21054.141602,0.000000,0.000000,0.000000,21069.632812,-2.555141,0.000000,0.000000
4,21052.195312,21053.752344,21053.752344,21053.752344,0.000000,0.000000,0.000000,21068.726641,-5.785522,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5730,21537.373047,21586.476842,21656.157813,21628.526870,21648.639453,21772.101192,21525.177714,21705.709504,-32.376104,35.357591,21671.883496
5731,21523.199219,21564.999163,21648.763672,21627.649246,21639.780371,21772.220862,21507.339880,21694.580176,-40.558197,30.294057,21666.162207
5732,21460.732422,21536.938895,21639.052188,21624.435685,21626.804883,21775.074378,21478.535388,21675.812043,-45.561511,32.344131,21659.956592
5733,21476.566406,21527.107422,21632.243984,21621.504774,21614.947559,21771.233125,21458.661992,21658.545361,-48.829209,32.644520,21654.082178


In [557]:
#Loads model

model = tf.keras.models.load_model('Classifier_NN_hourly_v2.0')

#Makes the prediction

y_pred = model.predict(df.iloc[-1:].to_numpy().reshape(1,-1))

#Makes the prediction

print(type(y_pred[0]))

df.iloc[-1:]

<class 'numpy.ndarray'>


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
5734,21477.091797,21513.00865,21624.472734,21619.174262,21605.454102,21770.755158,21440.153045,21642.660013,-50.791002,32.745812,21647.743213
